<a href="https://colab.research.google.com/github/pikey-msc/RiesgosFinancieros/blob/master/2022-2/Insumos/RH_acciones_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Riesgo de Mercado (Simulación Histórica)

Existen trés métodos principales para medir el riesgo de mercado
* Simulación histórica
* Paramétrico
* Simulación de Monte Carlo



   **Pérdidas y Ganancias simulación histórica (sin alisado)**
   
Un enfoque más claro es suponer que tenemos:
1.   Una matriz $X_{(n+1)\times m}$ de $m$ factores de riesgo y $n+1$ observaciones. 
2.   Denotemos el vector de precios actual como $X_{00}:=(x_{0,1},x_{0,2},\ldots,x_{0,m})$. 
3.   Sea $r$ el número de instrumentos de un portafolio, entonces cada instrumento tiene una función de valuación $f_i\colon A_i \to R$ para todo $x\in X$, $i=1,\ldots, r$, donde $A_i\subset X_i$ con $\#(A_i)\leq \#(X_i)$.
4.   Sea $r$ el número de instrumentos de un portafolio, entonces cada instrumento tiene una función de valuación $f_i\colon A_i \to R$ para todo $x\in X$, $i=1,\ldots, r$, donde $A_i\subset X_i$ con $\#(A_i)\leq \#(X_i)$.
5.   Sea $M_{1\times r}=(m_1,\ldots,m_r)$ el vector de posiciones nominales de cada instrumento, es decir, el número de contratos que se tienen por instrumento $m_i\in R$ $(i=1,\ldots,r)$.

In [1]:
!pip install yfinance
import yfinance as yf
import pandas as pd


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.1 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [2]:
start = '2020-01-01'

symbol = ['AAPL','MSFT','SQ','AMZN']
stock_data = yf.download(symbol,start)
# stock_data = stock_data.history(start=start)#, end=end) 
# symbol1='APPL'
# stock_data = yf.Ticker(symbol)
# stock_data = stock_data.history(start=start)#, end=end) 


[*********************100%***********************]  4 of 4 completed



##Paso 1.
 (Generación de rendimientos) Construir ${\Delta X}_{n\times m}$ que es la matriz de diferencias basados en el operador $T_j$, es decir
		$${\Delta X}_t=\Big[ T_j(\frac{x_{t,1}}{x_{t+1,1}}), T_j(\frac{x_{t,2}}{x_{t+1,2}}),\ldots, T_j(\frac{x_{t,m}}{x_{t+1,m}})\Big]\quad t=0,1,\ldots, n-1.$$

In [3]:
# yf.download?
print(stock_data.Close.head(5))

import numpy as np
X=stock_data.Close
X0=stock_data.Close.tail(1)
DeltaX=np.log(stock_data.Close/stock_data.Close.shift(1))

print(DeltaX.head())
print(X0)


                 AAPL       AMZN        MSFT         SQ
Date                                                   
2020-01-02  75.087502  94.900497  160.619995  63.830002
2020-01-03  74.357498  93.748497  158.619995  63.000000
2020-01-06  74.949997  95.143997  159.029999  62.570000
2020-01-07  74.597504  95.343002  157.580002  64.589996
2020-01-08  75.797501  94.598503  160.089996  67.599998
                AAPL      AMZN      MSFT        SQ
Date                                              
2020-01-02       NaN       NaN       NaN       NaN
2020-01-03 -0.009770 -0.012213 -0.012530 -0.013089
2020-01-06  0.007937  0.014776  0.002581 -0.006849
2020-01-07 -0.004714  0.002089 -0.009160  0.031774
2020-01-08  0.015958 -0.007839  0.015803  0.045548
                  AAPL        AMZN        MSFT     SQ
Date                                                 
2022-09-12  163.429993  136.449997  266.649994  75.25


In [4]:
import pandas as pd
M=np.array([1000,5000,2000,100])
print(M)

#Valoración

Y0=X0.squeeze()*M

Y0



[1000 5000 2000  100]


AAPL    163429.992676
AMZN    682249.984741
MSFT    533299.987793
SQ        7525.000000
Name: 2022-09-12 00:00:00, dtype: float64

##Paso 2. 
Construir ${X^\ast}_{n\times m}$ que es la matriz de factores de riesgo simulada basada en el vector de precios actual $X_{00}$
		$$X_{t}^\ast=\Big[x_{0,1}T^{-1}_j(\Delta x_{t,1}), x_{0,2}T^{-1}_j(\Delta x_{t,2}),\ldots, x_{0,m}T^{-1}_j(\Delta x_{t,m})\Big]\quad (t=1,\ldots,n\quad j=1,2),$$
		donde $T_1(x)=\ln(x)$, $T_2(x)=x-1$.



In [5]:
X_s=np.exp(DeltaX.squeeze())*X0.squeeze()
X_s.head

<bound method NDFrame.head of                   AAPL        AMZN        MSFT         SQ
Date                                                     
2020-01-02         NaN         NaN         NaN        NaN
2020-01-03  161.841120  134.793626  263.329735  74.271500
2020-01-06  164.732243  138.481133  267.339235  74.736389
2020-01-07  162.661374  136.735399  264.218744  77.679355
2020-01-08  166.058975  135.384508  270.897297  78.756776
...                ...         ...         ...        ...
2022-09-06  162.087395  134.951838  263.723782  74.081488
2022-09-07  164.942360  140.096304  271.746085  78.153978
2022-09-08  161.858147  136.808312  267.094248  78.323463
2022-09-09  166.508973  140.076187  272.776799  79.194261
2022-09-12  169.723348  139.705868  268.858132  76.222404

[679 rows x 4 columns]>

## Paso 3. 
Construcción de la matriz de reevaluación del portafolio basada en los escenarios históricos $Y_{n\times r}$ de todos los instrumentos financieros, es decir
		$$Y_t=[m_1f_1(X_{t}^\ast), m_2f_2(X_{t}^\ast),\ldots, m_rf_r(X_{t}^\ast)]\quad (t=1,\ldots,n).$$
    


In [6]:
Y_s=X_s*M.squeeze()
Y_s.head


<bound method NDFrame.head of                      AAPL           AMZN           MSFT           SQ
Date                                                                
2020-01-02            NaN            NaN            NaN          NaN
2020-01-03  161841.120482  673968.128532  526659.469750  7427.150030
2020-01-06  164732.242934  692405.667328  534678.470675  7473.638852
2020-01-07  162661.373896  683676.992738  528437.487883  7767.935509
2020-01-08  166058.974650  676922.540042  541794.593862  7875.677618
...                   ...            ...            ...          ...
2022-09-06  162087.394676  674759.191833  527447.563838  7408.148792
2022-09-07  164942.360424  700481.521544  543492.169384  7815.397762
2022-09-08  161858.147469  684041.558068  534188.496034  7832.346282
2022-09-09  166508.973401  700380.937079  545553.598207  7919.426064
2022-09-12  169723.348381  698529.341573  537716.263649  7622.240450

[679 rows x 4 columns]>

## Paso 4. 
Construcción de la matriz de pérdidas y ganancias del portafolio basada en los escenarios históricos $\Delta Y_{n\times r}$ de todos los instrumentos financieros, es decir
		$$\Delta Y_t=Y_t-Y_{0}=[m_1(f_1(X_{00})-f_1(X_{t}^\ast)), m_2( f_2(X_{00})-f_2(X_{t}^\ast)),\ldots, m_r(f_r(X_{00})-f_r(X_{t}^\ast))],$$
		para $(t=1,\ldots,n)$. Se puede obtener el vector de pérdidas totales ${\Delta YT}_{n\times m}$ muy fácilmente, esto es $\Delta YT=\sum_{k=1}^r Y_{kt}$ $(t=1,\ldots,n)$, incluso se puede hacer lo mismo por tipo de riesgo.


In [7]:
PL=Y_s-Y0

cols=PL.shape[1]

PL['Total']=PL.sum(axis=1)
PL.head

<bound method NDFrame.head of                    AAPL          AMZN          MSFT          SQ         Total
Date                                                                         
2020-01-02          NaN           NaN           NaN         NaN      0.000000
2020-01-03 -1588.872194  -8281.856209  -6640.518043  -97.849970 -16609.096415
2020-01-06  1302.250258  10155.682587   1378.482882  -51.361148  12785.054579
2020-01-07  -768.618779   1427.007997  -4862.499910  242.935509  -3961.175184
2020-01-08  2628.981974  -5327.444699   8494.606069  350.677618   6146.820962
...                 ...           ...           ...         ...           ...
2022-09-06 -1342.598000  -7490.792908  -5852.423955 -116.851208 -14802.666071
2022-09-07  1512.367748  18231.536802  10192.181591  290.397762  30226.483903
2022-09-08 -1571.845207   1791.573326    888.508241  307.346282   1415.582643
2022-09-09  3078.980725  18130.952338  12253.610414  394.426064  33857.969541
2022-09-12  6293.355705  16279.356

##Paso 5. 
Obtenerla medida de riesgo basado en un nivel de confianza de la matriz ${\Delta Y}_{\cdot k}$ $(k=1,\ldots,r)$ y $\Delta YT$.

Las pérdidas y ganancias vía simulación histórica sin alisado es donde se ponderan los escenarios históricos con un mismo peso para todos, es decir cada escenario tiene una probabilidad de ocurrencia de $1/n$.

In [8]:
alpha = 0.98 #@param {type:"slider", min:0.8, max:1, step:0.001}

###Valor en Riesgo (VaR)

Es un cuantil a un $\alpha$ de confianza de la distribución de pérdidas y ganancias, generalmente se expresa en términos monetarios.

#VaR

El valor en riesgo (VaR) es una estadística utilizada para tratar de cuantificar el nivel de riesgo financiero dentro de una empresa o cartera durante un período de tiempo específico. VaR proporciona una estimación de la pérdida máxima de una posición o cartera determinada durante un período de tiempo, y puede calcularla a través de varios niveles de confianza.

Estimar el riesgo de una cartera es importante para el crecimiento del capital a largo plazo y la gestión del riesgo, en particular dentro de empresas o instituciones más grandes. El VaR generalmente se enmarca como algo como esto:

“Tenemos un VaR de cartera de 250.000 USD para el próximo mes al 95% de confianza”
Esto significa que, con un 95% de confianza, podemos decir que la pérdida de la cartera no superará los 250.000 USD en un mes.
En esta publicación, lo guiaré a través de los pasos para calcular esta métrica en una cartera de acciones.

In [9]:
VaR=PL.quantile(1-alpha, numeric_only=False)

print(VaR)

print(sum(Y0))

AAPL     -7658.126980
AMZN    -35838.394058
MSFT    -23086.524766
SQ        -670.420290
Total   -62256.289424
Name: 0.020000000000000018, dtype: float64
1386504.965209961


### Valor en Riesgo sin Cuantil

In [27]:
esc_var=np.round(PL.shape[0]*(1-alpha),0)
print(esc_var)
VaR_r=PL.rank()
VaR_nq=[]

for i in range(0,PL.shape[1]):
  VaR_nq_t=PL[VaR_r.iloc[:,i]==esc_var].iloc[:,i]
  VaR_nq.append(VaR_nq_t)

print(VaR_nq)

14.0
[Date
2020-02-24   -7762.987663
Name: AAPL, dtype: float64, Date
2020-03-09   -36059.526384
Name: AMZN, dtype: float64, Date
2020-07-23   -23195.733025
Name: MSFT, dtype: float64, Date
2021-03-18   -676.874408
Name: SQ, dtype: float64, Date
2022-03-07   -63015.432899
Name: Total, dtype: float64]


###Valor en Riesgo Condicional o "Expected Shortfall"

Es el promedio de las pérdidas que son más severas que el Valor en Riesgo ($VaR_\alpha$) a un $\alpha$ de confianza, es decir:
$$E(x|x\le VaR_\alpha)$$


In [ ]:
def VaR_Cond(x , VaR ):
  return np.mean(x[x <= VaR])

In [ ]:

VaR_C=[]

for i in range(0,PL.shape[1]):
  VC=VaR_Cond(PL.iloc[:,i],VaR[i])
  VaR_C.append(VC)


In [ ]:

print(VaR)
print(VaR_C)

pd.concat([VaR,pd.DataFrame(VaR_C).squeeze()], axis=1, ignore_index=True)

AAPL      -7298.874401
AMZN    -630538.408388
MSFT     -23795.190363
SQ         -813.530388
Total   -677462.575261
Name: 0.020000000000000018, dtype: float64
[-11261.958141897987, -846332.0609951812, -36937.994447413366, -1200.8464660778375, -881792.5615757715]


,0,1
AAPL,-7298.874401,NaN
AMZN,-630538.408388,NaN
MSFT,-23795.190363,NaN
SQ,-813.530388,NaN
Total,-677462.575261,NaN
0,NaN,-11261.958142
1,NaN,-846332.060995
2,NaN,-36937.994447
3,NaN,-1200.846466
4,NaN,-881792.561576
